## Notebook d'exploration - Projet 4 - David Depouez

### 1. Importation et selection

Importation des modules et lecture du fichier de données

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import missingno as msno

In [2]:
outputRawfilename = "-Raw.csv"
outputFullfilename = "-Full.csv"

outputRawfilename = 'Cleaned-' + Version + outputRawfilename
outputFullfilename = 'Cleaned-' + Version + outputFullfilename

print("Output Raw DataFrame filename will be :",outputRawfilename)
print("Output Formated Full DataFrame filename will be :",outputFullfilename)

Output Raw DataFrame filename will be : Cleaned-V4-Raw.csv
Output Formated Full DataFrame filename will be : Cleaned-V4-Full.csv


In [3]:
path = "./data/source/"

isGeolocationIncluded = False

filesList = ["olist_orders_dataset.csv","olist_order_items_dataset.csv","olist_order_payments_dataset.csv",
"olist_customers_dataset.csv", "olist_sellers_dataset.csv", "olist_products_dataset.csv","olist_order_reviews_dataset.csv",
"olist_geolocation_dataset.csv","product_category_name_translation.csv"]

dataBaseNumber = len(filesList)
dfList = []

for i,name in enumerate(filesList):
    a = pd.read_csv(path+filesList[i])
    dfList.append(a)

Une fonction pour analyser des colonnes d'un dataframe

In [4]:
# function to dislay and test some cols values
def checkValues(df,col,value,isDump = True):
    mask = df[col] == value
    if isDump == True:
        display(mask.value_counts())
    return df.loc[mask == True,:]

Une fonction pour avoir une liste des colonnes complementaires dans un dataframe

In [5]:
def getCompCols(df,cols):
    ### Get complements colons of cols for a dataframe df ###
    ComplementCols =  [name for i,name in enumerate(df.columns) if name not in cols]
    return ComplementCols

Une fonction pour mettre en forme le dataframe brut en fonction d'une date de fin

In [6]:
import datetime
from datetime import timedelta

year = timedelta(days=365)
month = timedelta(days=30)

def extract(df_to_extract, end_date):
    
    print("Calling extract with date : ",end_date)
    maskHere = (df_to_extract['order_purchase_timestamp'] <= end_date)
    df_extract_reducted = df_to_extract.loc[maskHere == True,:]
    
    number = len(df_extract_reducted)
    print("Number of observation selected",number)
    
    # now group the dataframe by customer id and do compute the features values
    grouped = df_extract_reducted.groupby(by= ['customer_unique_id'] ,dropna=False)
    df_extract_reducted = grouped.agg({'order_id':'count','order_purchase_timestamp':'max','retard':'mean','delai':'mean',
                           'payment_value':'sum', 'review_score':'mean', 'word_number':'sum'})
    df_extract_reducted.rename(columns={'order_id':'Frequency','order_purchase_timestamp':'Recency','payment_value':'Monetary'},inplace=True)
    df_extract_reducted = df_extract_reducted.reset_index()
    
    # compute recency as number of days since the last command from today
    df_extract_reducted['Recency'] =  df_extract_reducted['Recency'].apply(lambda x: (pd.to_datetime('today') - x).days)
    
    df_extract_reducted.drop('customer_unique_id',axis=1,inplace=True)
    
    return df_extract_reducted

Check for Orders

In [7]:
index = 0
print("Shape for {} is {}".format(filesList[index].strip('olist'),dfList[index].shape))
print("Cols are {}".format(dfList[index].columns))
ordersColsToKeep = ['customer_id','order_id']
ordersColsToKeep.append('order_status')
ordersColsToKeep.append('order_purchase_timestamp')
ordersColsToKeep.append('order_delivered_customer_date')
ordersColsToKeep.append('order_estimated_delivery_date')
dfTemp = dfList[index].copy()
dfTemp.head()

Shape for _orders_dataset.csv is (99441, 8)
Cols are Index(['order_id', 'customer_id', 'order_status', 'order_purchase_timestamp',
       'order_approved_at', 'order_delivered_carrier_date',
       'order_delivered_customer_date', 'order_estimated_delivery_date'],
      dtype='object')


,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13 00:00:00
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04 00:00:00
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2017-11-18 19:28:06,2017-11-18 19:45:59,2017-11-22 13:39:59,2017-12-02 00:28:42,2017-12-15 00:00:00
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,2018-02-13 21:18:39,2018-02-13 22:20:29,2018-02-14 19:46:34,2018-02-16 18:17:02,2018-02-26 00:00:00


In [8]:
ColsToThrow = getCompCols(dfTemp,ordersColsToKeep)
dfTemp = dfTemp.drop(ColsToThrow,axis=1)

In [9]:
dfTemp.nunique()

order_id                         99441
customer_id                      99441
order_status                         8
order_purchase_timestamp         98875
order_delivered_customer_date    95664
order_estimated_delivery_date      459
dtype: int64

In [10]:
dfTemp.isna().sum()

order_id                            0
customer_id                         0
order_status                        0
order_purchase_timestamp            0
order_delivered_customer_date    2965
order_estimated_delivery_date       0
dtype: int64

In [11]:
dfTemp.duplicated().value_counts()

False    99441
dtype: int64

Check for Orders Items

In [12]:
index = 1
print("Shape for {} is {}".format(filesList[index].strip('olist'),dfList[index].shape))
print("Cols are {}".format(dfList[index].columns))
ColsToKeep = ['product_id','order_id']
ColsToKeep.append('price')
#ColsToKeep.append('freight_value')
#ColsToKeep.append('order_item_id')
dfTemp = dfList[index].copy()
dfTemp.head()

Shape for _order_items_dataset.csv is (112650, 7)
Cols are Index(['order_id', 'order_item_id', 'product_id', 'seller_id',
       'shipping_limit_date', 'price', 'freight_value'],
      dtype='object')


,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value
0,00010242fe8c5a6d1ba2dd792cb16214,1,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,2017-09-19 09:45:35,58.90,13.29
1,00018f77f2f0320c557190d7a144bdd3,1,e5f2d52b802189ee658865ca93d83a8f,dd7ddc04e1b6c2c614352b383efe2d36,2017-05-03 11:05:13,239.90,19.93
2,000229ec398224ef6ca0657da4fc703e,1,c777355d18b72b67abbeef9df44fd0fd,5b51032eddd242adc84c38acab88f23d,2018-01-18 14:48:30,199.00,17.87
3,00024acbcdf0a6daa1e931b038114c75,1,7634da152a4610f1595efa32f14722fc,9d7a1d34a5052409006425275ba1c2b4,2018-08-15 10:10:18,12.99,12.79
4,00042b26cf59d7ce69dfabb4e55b4fd9,1,ac6c3623068f30de03045865e4e10089,df560393f3a51e74553ab94004ba5c87,2017-02-13 13:57:51,199.90,18.14


On enleve la colonne order_item_id afin de trouver les doublons et de pouvoir compter le nombre d'achats identiques sur une commande

In [13]:
ColsToThrow = getCompCols(dfTemp,ColsToKeep)
dfTemp = dfTemp.drop(ColsToThrow,axis=1)
dfTemp

,order_id,product_id,price
0,00010242fe8c5a6d1ba2dd792cb16214,4244733e06e7ecb4970a6e2683c13e61,58.90
1,00018f77f2f0320c557190d7a144bdd3,e5f2d52b802189ee658865ca93d83a8f,239.90
2,000229ec398224ef6ca0657da4fc703e,c777355d18b72b67abbeef9df44fd0fd,199.00
3,00024acbcdf0a6daa1e931b038114c75,7634da152a4610f1595efa32f14722fc,12.99
4,00042b26cf59d7ce69dfabb4e55b4fd9,ac6c3623068f30de03045865e4e10089,199.90
...,...,...,...
112645,fffc94f6ce00a00581880bf54a75a037,4aa6014eceb682077f9dc4bffebc05b0,299.99
112646,fffcd46ef2263f404302a634eb57f7eb,32e07fd915822b0765e448c4dd74c828,350.00
112647,fffce4705a9662cd70adb13d4a31832d,72a30483855e2eafc67aee5dc2560482,99.90
112648,fffe18544ffabc95dfada21779c9644f,9c422a519119dcad7575db5af1ba540e,55.99


In [16]:
dfTemp = dfTemp.groupby(by=['order_id','product_id'],dropna=False).sum()
dfTemp = dfTemp.reset_index()
dfTemp

,order_id,product_id,price
0,00010242fe8c5a6d1ba2dd792cb16214,4244733e06e7ecb4970a6e2683c13e61,58.90
1,00018f77f2f0320c557190d7a144bdd3,e5f2d52b802189ee658865ca93d83a8f,239.90
2,000229ec398224ef6ca0657da4fc703e,c777355d18b72b67abbeef9df44fd0fd,199.00
3,00024acbcdf0a6daa1e931b038114c75,7634da152a4610f1595efa32f14722fc,12.99
4,00042b26cf59d7ce69dfabb4e55b4fd9,ac6c3623068f30de03045865e4e10089,199.90
...,...,...,...
102420,fffc94f6ce00a00581880bf54a75a037,4aa6014eceb682077f9dc4bffebc05b0,299.99
102421,fffcd46ef2263f404302a634eb57f7eb,32e07fd915822b0765e448c4dd74c828,350.00
102422,fffce4705a9662cd70adb13d4a31832d,72a30483855e2eafc67aee5dc2560482,99.90
102423,fffe18544ffabc95dfada21779c9644f,9c422a519119dcad7575db5af1ba540e,55.99


In [17]:
dfTemp.duplicated().value_counts()

False    102425
dtype: int64

In [18]:
dfTemp.nunique()

order_id      98666
product_id    32951
price          6981
dtype: int64

In [19]:
dfTemp.isna().sum()

order_id      0
product_id    0
price         0
dtype: int64

Premier Merge 

In [20]:
df = pd.merge(dfList[0].loc[:,ordersColsToKeep] ,dfTemp.loc[:,ColsToKeep], on='order_id',how='outer')
print("Shape for df is: {}".format(df.shape))
df.head()

Shape for df is: (103200, 8)


,customer_id,order_id,order_status,order_purchase_timestamp,order_delivered_customer_date,order_estimated_delivery_date,product_id,price
0,9ef432eb6251297304e76186b10a928d,e481f51cbdc54678b7cc49136f2d6af7,delivered,2017-10-02 10:56:33,2017-10-10 21:25:13,2017-10-18 00:00:00,87285b34884572647811a353c7ac498a,29.99
1,b0830fb4747a6c6d20dea0b8c802d7ef,53cdb2fc8bc7dce0b6741e2150273451,delivered,2018-07-24 20:41:37,2018-08-07 15:27:45,2018-08-13 00:00:00,595fac2a385ac33a80bd5114aec74eb8,118.70
2,41ce2a54c0b03bf3443c3d931a367089,47770eb9100c2d0c44946d9cf07ec65d,delivered,2018-08-08 08:38:49,2018-08-17 18:06:29,2018-09-04 00:00:00,aa4383b373c6aca5d8797843e5594415,159.90
3,f88197465ea7920adcdbec7375364d82,949d5b44dbf5de918fe9c16f97b45f8a,delivered,2017-11-18 19:28:06,2017-12-02 00:28:42,2017-12-15 00:00:00,d0b61bfb1de832b15ba9d266ca96e5b0,45.00
4,8ab97904e6daea8866dbdbc4fb7aad2c,ad21c59c0840e6cb83a9ceb5573f8159,delivered,2018-02-13 21:18:39,2018-02-16 18:17:02,2018-02-26 00:00:00,65266b2da20d04dbe00c5c2d3bb7859e,19.90


In [21]:
df.nunique()

customer_id                      99441
order_id                         99441
order_status                         8
order_purchase_timestamp         98875
order_delivered_customer_date    95664
order_estimated_delivery_date      459
product_id                       32951
price                             6981
dtype: int64

In [22]:
df.isna().sum()

customer_id                         0
order_id                            0
order_status                        0
order_purchase_timestamp            0
order_delivered_customer_date    3005
order_estimated_delivery_date       0
product_id                        775
price                             775
dtype: int64

In [23]:
df.duplicated().value_counts()

False    103200
dtype: int64

Check for orders payments

In [24]:
index = 2
print("Shape for {} is {}".format(filesList[index].strip('olist'),dfList[index].shape))
print("Cols are {}".format(dfList[index].columns))
ColsToKeep = ['order_id']
#ColsToKeep.append('payment_sequential')
#ColsToKeep.append('payment_installments')
#ColsToKeep.append('payment_type')
ColsToKeep.append('payment_value')
dfTemp = dfList[index].copy()
dfTemp.head()

Shape for _order_payments_dataset.csv is (103886, 5)
Cols are Index(['order_id', 'payment_sequential', 'payment_type',
       'payment_installments', 'payment_value'],
      dtype='object')


,order_id,payment_sequential,payment_type,payment_installments,payment_value
0,b81ef226f3fe1789b1e8b2acac839d17,1,credit_card,8,99.33
1,a9810da82917af2d9aefd1278f1dcfa0,1,credit_card,1,24.39
2,25e8ea4e93396b6fa0d3dd708e76c1bd,1,credit_card,1,65.71
3,ba78997921bbcdc1373bb41e913ab953,1,credit_card,8,107.78
4,42fdf880ba16b47b59251dd489d4441a,1,credit_card,2,128.45


In [25]:
ColsToThrow = getCompCols(dfTemp,ColsToKeep)
dfTemp = dfTemp.drop(ColsToThrow,axis=1)
dfTemp

,order_id,payment_value
0,b81ef226f3fe1789b1e8b2acac839d17,99.33
1,a9810da82917af2d9aefd1278f1dcfa0,24.39
2,25e8ea4e93396b6fa0d3dd708e76c1bd,65.71
3,ba78997921bbcdc1373bb41e913ab953,107.78
4,42fdf880ba16b47b59251dd489d4441a,128.45
...,...,...
103881,0406037ad97740d563a178ecc7a2075c,363.31
103882,7b905861d7c825891d6347454ea7863f,96.80
103883,32609bbb3dd69b3c066a6860554a77bf,47.77
103884,b8b61059626efa996a60be9bb9320e10,369.54


In [26]:
dfTemp = dfTemp.groupby(by=['order_id'],dropna=False).sum()
dfTemp = dfTemp.reset_index()
dfTemp

,order_id,payment_value
0,00010242fe8c5a6d1ba2dd792cb16214,72.19
1,00018f77f2f0320c557190d7a144bdd3,259.83
2,000229ec398224ef6ca0657da4fc703e,216.87
3,00024acbcdf0a6daa1e931b038114c75,25.78
4,00042b26cf59d7ce69dfabb4e55b4fd9,218.04
...,...,...
99435,fffc94f6ce00a00581880bf54a75a037,343.40
99436,fffcd46ef2263f404302a634eb57f7eb,386.53
99437,fffce4705a9662cd70adb13d4a31832d,116.85
99438,fffe18544ffabc95dfada21779c9644f,64.71


In [27]:
dfTemp.nunique()

order_id         99440
payment_value    28029
dtype: int64

In [28]:
dfTemp.isna().sum()

order_id         0
payment_value    0
dtype: int64

In [29]:
dfTemp.duplicated().value_counts()

False    99440
dtype: int64

Merge

In [30]:
df = pd.merge(df,dfTemp.loc[:,ColsToKeep], on='order_id',how='outer')
print("Shape for df is: {}".format(df.shape))
df.head()

Shape for df is: (103200, 9)


,customer_id,order_id,order_status,order_purchase_timestamp,order_delivered_customer_date,order_estimated_delivery_date,product_id,price,payment_value
0,9ef432eb6251297304e76186b10a928d,e481f51cbdc54678b7cc49136f2d6af7,delivered,2017-10-02 10:56:33,2017-10-10 21:25:13,2017-10-18 00:00:00,87285b34884572647811a353c7ac498a,29.99,38.71
1,b0830fb4747a6c6d20dea0b8c802d7ef,53cdb2fc8bc7dce0b6741e2150273451,delivered,2018-07-24 20:41:37,2018-08-07 15:27:45,2018-08-13 00:00:00,595fac2a385ac33a80bd5114aec74eb8,118.70,141.46
2,41ce2a54c0b03bf3443c3d931a367089,47770eb9100c2d0c44946d9cf07ec65d,delivered,2018-08-08 08:38:49,2018-08-17 18:06:29,2018-09-04 00:00:00,aa4383b373c6aca5d8797843e5594415,159.90,179.12
3,f88197465ea7920adcdbec7375364d82,949d5b44dbf5de918fe9c16f97b45f8a,delivered,2017-11-18 19:28:06,2017-12-02 00:28:42,2017-12-15 00:00:00,d0b61bfb1de832b15ba9d266ca96e5b0,45.00,72.20
4,8ab97904e6daea8866dbdbc4fb7aad2c,ad21c59c0840e6cb83a9ceb5573f8159,delivered,2018-02-13 21:18:39,2018-02-16 18:17:02,2018-02-26 00:00:00,65266b2da20d04dbe00c5c2d3bb7859e,19.90,28.62


In [31]:
df.nunique()

customer_id                      99441
order_id                         99441
order_status                         8
order_purchase_timestamp         98875
order_delivered_customer_date    95664
order_estimated_delivery_date      459
product_id                       32951
price                             6981
payment_value                    28029
dtype: int64

In [32]:
df.duplicated().value_counts()

False    103200
dtype: int64

In [33]:
df.isna().sum()

customer_id                         0
order_id                            0
order_status                        0
order_purchase_timestamp            0
order_delivered_customer_date    3005
order_estimated_delivery_date       0
product_id                        775
price                             775
payment_value                       1
dtype: int64

Check for customers

In [34]:
index = 3
print("Shape for {} is {}".format(filesList[index].strip('olist'),dfList[index].shape))
print("Cols are {}".format(dfList[index].columns))
ColsToKeep = ['customer_id']
ColsToKeep.append('customer_unique_id')
#ColsToKeep.append('customer_zip_code_prefix')
#ColsToKeep.append('customer_city')
dfTemp = dfList[index].copy()
dfTemp.head()

Shape for _customers_dataset.csv is (99441, 5)
Cols are Index(['customer_id', 'customer_unique_id', 'customer_zip_code_prefix',
       'customer_city', 'customer_state'],
      dtype='object')


,customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state
0,06b8999e2fba1a1fbc88172c00ba8bc7,861eff4711a542e4b93843c6dd7febb0,14409,franca,SP
1,18955e83d337fd6b2def6b18a428ac77,290c77bc529b7ac935b93aa66c333dc3,9790,sao bernardo do campo,SP
2,4e7b3e00288586ebd08712fdd0374a03,060e732b5b29e8181a18229c7b0b2b5e,1151,sao paulo,SP
3,b2b6027bc5c5109e529d4dc6358b12c3,259dac757896d24d7702b9acbbff3f3c,8775,mogi das cruzes,SP
4,4f2d8ab171c80ec8364f7c12e35b23ad,345ecd01c38d18a9036ed96c73b8d066,13056,campinas,SP


In [35]:
dfTemp.nunique()

customer_id                 99441
customer_unique_id          96096
customer_zip_code_prefix    14994
customer_city                4119
customer_state                 27
dtype: int64

In [36]:
dfTemp.isna().sum()

customer_id                 0
customer_unique_id          0
customer_zip_code_prefix    0
customer_city               0
customer_state              0
dtype: int64

Merge

In [37]:
df = pd.merge(df,dfTemp.loc[:,ColsToKeep], on='customer_id',how='outer')
print("Shape for df is: {}".format(df.shape))
df.head()

Shape for df is: (103200, 10)


,customer_id,order_id,order_status,order_purchase_timestamp,order_delivered_customer_date,order_estimated_delivery_date,product_id,price,payment_value,customer_unique_id
0,9ef432eb6251297304e76186b10a928d,e481f51cbdc54678b7cc49136f2d6af7,delivered,2017-10-02 10:56:33,2017-10-10 21:25:13,2017-10-18 00:00:00,87285b34884572647811a353c7ac498a,29.99,38.71,7c396fd4830fd04220f754e42b4e5bff
1,b0830fb4747a6c6d20dea0b8c802d7ef,53cdb2fc8bc7dce0b6741e2150273451,delivered,2018-07-24 20:41:37,2018-08-07 15:27:45,2018-08-13 00:00:00,595fac2a385ac33a80bd5114aec74eb8,118.70,141.46,af07308b275d755c9edb36a90c618231
2,41ce2a54c0b03bf3443c3d931a367089,47770eb9100c2d0c44946d9cf07ec65d,delivered,2018-08-08 08:38:49,2018-08-17 18:06:29,2018-09-04 00:00:00,aa4383b373c6aca5d8797843e5594415,159.90,179.12,3a653a41f6f9fc3d2a113cf8398680e8
3,f88197465ea7920adcdbec7375364d82,949d5b44dbf5de918fe9c16f97b45f8a,delivered,2017-11-18 19:28:06,2017-12-02 00:28:42,2017-12-15 00:00:00,d0b61bfb1de832b15ba9d266ca96e5b0,45.00,72.20,7c142cf63193a1473d2e66489a9ae977
4,8ab97904e6daea8866dbdbc4fb7aad2c,ad21c59c0840e6cb83a9ceb5573f8159,delivered,2018-02-13 21:18:39,2018-02-16 18:17:02,2018-02-26 00:00:00,65266b2da20d04dbe00c5c2d3bb7859e,19.90,28.62,72632f0f9dd73dfee390c9b22eb56dd6


In [38]:
df.nunique()

customer_id                      99441
order_id                         99441
order_status                         8
order_purchase_timestamp         98875
order_delivered_customer_date    95664
order_estimated_delivery_date      459
product_id                       32951
price                             6981
payment_value                    28029
customer_unique_id               96096
dtype: int64

In [39]:
df.duplicated().value_counts()

False    103200
dtype: int64

Check for Sellers

In [40]:
index = 4
print("Shape for {} is {}".format(filesList[index].strip('olist'),dfList[index].shape))
print("Cols are {}".format(dfList[index].columns))
ColsToKeep = ['seller_id']
ColsToKeep.append('seller_zip_code_prefix')
dfTemp = dfList[index].copy()
dfTemp.head()

Shape for _sellers_dataset.csv is (3095, 4)
Cols are Index(['seller_id', 'seller_zip_code_prefix', 'seller_city', 'seller_state'], dtype='object')


,seller_id,seller_zip_code_prefix,seller_city,seller_state
0,3442f8959a84dea7ee197c632cb2df15,13023,campinas,SP
1,d1b65fc7debc3361ea86b5f14c68d2e2,13844,mogi guacu,SP
2,ce3ad9de960102d0677a81f5d0bb7b2d,20031,rio de janeiro,RJ
3,c0f3eea2e14555b6faeea3dd58c1b1c3,4195,sao paulo,SP
4,51a04a8a6bdcb23deccc82b0b80742cf,12914,braganca paulista,SP


Merge

In [41]:
useSellerTab = False
if useSellerTab == True:
    df = pd.merge(df,dfTemp.loc[:,ColsToKeep], on='seller_id',how='left')
    print("Shape for df is: {}".format(df.shape))
    df.head()

In [42]:
df.nunique()

customer_id                      99441
order_id                         99441
order_status                         8
order_purchase_timestamp         98875
order_delivered_customer_date    95664
order_estimated_delivery_date      459
product_id                       32951
price                             6981
payment_value                    28029
customer_unique_id               96096
dtype: int64

In [43]:
df.isna().sum()

customer_id                         0
order_id                            0
order_status                        0
order_purchase_timestamp            0
order_delivered_customer_date    3005
order_estimated_delivery_date       0
product_id                        775
price                             775
payment_value                       1
customer_unique_id                  0
dtype: int64

In [44]:
df.duplicated().value_counts()

False    103200
dtype: int64

Check for products

In [45]:
index = 5
print("Shape for {} is {}".format(filesList[index].strip('olist'),dfList[index].shape))
print("Cols are {}".format(dfList[index].columns))
ColsToKeep = ['product_id']
ColsToKeep.append('product_category_name')
#ColsToKeep.append('product_photos_qty')
#ColsToKeep.append('product_description_lenght')
dfTemp = dfList[index].copy()
dfTemp.head()

Shape for _products_dataset.csv is (32951, 9)
Cols are Index(['product_id', 'product_category_name', 'product_name_lenght',
       'product_description_lenght', 'product_photos_qty', 'product_weight_g',
       'product_length_cm', 'product_height_cm', 'product_width_cm'],
      dtype='object')


,product_id,product_category_name,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm
0,1e9e8ef04dbcff4541ed26657ea517e5,perfumaria,40.0,287.0,1.0,225.0,16.0,10.0,14.0
1,3aa071139cb16b67ca9e5dea641aaa2f,artes,44.0,276.0,1.0,1000.0,30.0,18.0,20.0
2,96bd76ec8810374ed1b65e291975717f,esporte_lazer,46.0,250.0,1.0,154.0,18.0,9.0,15.0
3,cef67bcfe19066a932b7673e239eb23d,bebes,27.0,261.0,1.0,371.0,26.0,4.0,26.0
4,9dc1a7de274444849c219cff195d0b71,utilidades_domesticas,37.0,402.0,4.0,625.0,20.0,17.0,13.0


In [46]:
ColsToThrow = getCompCols(dfTemp,ColsToKeep)
dfTemp = dfTemp.drop(ColsToThrow,axis=1)

In [47]:
dfTemp.nunique()

product_id               32951
product_category_name       73
dtype: int64

In [48]:
dfTemp.isna().sum()

product_id                 0
product_category_name    610
dtype: int64

In [49]:
dfTemp.duplicated().value_counts()

False    32951
dtype: int64

Merge

In [50]:
df = pd.merge(df,dfTemp.loc[:,ColsToKeep], on='product_id',how='left')
print("Shape for df is: {}".format(df.shape))
df.head()

Shape for df is: (103200, 11)


,customer_id,order_id,order_status,order_purchase_timestamp,order_delivered_customer_date,order_estimated_delivery_date,product_id,price,payment_value,customer_unique_id,product_category_name
0,9ef432eb6251297304e76186b10a928d,e481f51cbdc54678b7cc49136f2d6af7,delivered,2017-10-02 10:56:33,2017-10-10 21:25:13,2017-10-18 00:00:00,87285b34884572647811a353c7ac498a,29.99,38.71,7c396fd4830fd04220f754e42b4e5bff,utilidades_domesticas
1,b0830fb4747a6c6d20dea0b8c802d7ef,53cdb2fc8bc7dce0b6741e2150273451,delivered,2018-07-24 20:41:37,2018-08-07 15:27:45,2018-08-13 00:00:00,595fac2a385ac33a80bd5114aec74eb8,118.70,141.46,af07308b275d755c9edb36a90c618231,perfumaria
2,41ce2a54c0b03bf3443c3d931a367089,47770eb9100c2d0c44946d9cf07ec65d,delivered,2018-08-08 08:38:49,2018-08-17 18:06:29,2018-09-04 00:00:00,aa4383b373c6aca5d8797843e5594415,159.90,179.12,3a653a41f6f9fc3d2a113cf8398680e8,automotivo
3,f88197465ea7920adcdbec7375364d82,949d5b44dbf5de918fe9c16f97b45f8a,delivered,2017-11-18 19:28:06,2017-12-02 00:28:42,2017-12-15 00:00:00,d0b61bfb1de832b15ba9d266ca96e5b0,45.00,72.20,7c142cf63193a1473d2e66489a9ae977,pet_shop
4,8ab97904e6daea8866dbdbc4fb7aad2c,ad21c59c0840e6cb83a9ceb5573f8159,delivered,2018-02-13 21:18:39,2018-02-16 18:17:02,2018-02-26 00:00:00,65266b2da20d04dbe00c5c2d3bb7859e,19.90,28.62,72632f0f9dd73dfee390c9b22eb56dd6,papelaria


In [51]:
df.nunique()

customer_id                      99441
order_id                         99441
order_status                         8
order_purchase_timestamp         98875
order_delivered_customer_date    95664
order_estimated_delivery_date      459
product_id                       32951
price                             6981
payment_value                    28029
customer_unique_id               96096
product_category_name               73
dtype: int64

In [52]:
df.duplicated().value_counts()

False    103200
dtype: int64

In [53]:
df.isna().sum()

customer_id                         0
order_id                            0
order_status                        0
order_purchase_timestamp            0
order_delivered_customer_date    3005
order_estimated_delivery_date       0
product_id                        775
price                             775
payment_value                       1
customer_unique_id                  0
product_category_name            2235
dtype: int64

Check for reviews

In [54]:
index = 6
print("Shape for {} is {}".format(filesList[index].strip('olist'),dfList[index].shape))
print("Cols are {}".format(dfList[index].columns))
ColsToKeep = ['order_id']
#ColsToKeep.append('review_comment_message')
ColsToKeep.append('review_score')
dfTemp = dfList[index].copy()
dfTemp.head()

Shape for _order_reviews_dataset.csv is (99224, 7)
Cols are Index(['review_id', 'order_id', 'review_score', 'review_comment_title',
       'review_comment_message', 'review_creation_date',
       'review_answer_timestamp'],
      dtype='object')


,review_id,order_id,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp
0,7bc2406110b926393aa56f80a40eba40,73fc7af87114b39712e6da79b0a377eb,4,NaN,NaN,2018-01-18 00:00:00,2018-01-18 21:46:59
1,80e641a11e56f04c1ad469d5645fdfde,a548910a1c6147796b98fdf73dbeba33,5,NaN,NaN,2018-03-10 00:00:00,2018-03-11 03:05:13
2,228ce5500dc1d8e020d8d1322874b6f0,f9e4b658b201a9f2ecdecbb34bed034b,5,NaN,NaN,2018-02-17 00:00:00,2018-02-18 14:36:24
3,e64fb393e7b32834bb789ff8bb30750e,658677c97b385a9be170737859d3511b,5,NaN,Recebi bem antes do prazo estipulado.,2017-04-21 00:00:00,2017-04-21 22:02:06
4,f7c4243c7fe1938f181bec41a392bdeb,8e6bfb81e283fa7e4f11123a3fb894f1,5,NaN,Parabéns lojas lannister adorei comprar pela I...,2018-03-01 00:00:00,2018-03-02 10:26:53


on compte le nombre de mots par message

In [55]:
dfTemp['word_number'] = dfTemp['review_comment_message'].str.split().str.len()

si pas d'infos (NaN) on met 0 mots

In [56]:
dfTemp['word_number'] = dfTemp['word_number'].fillna(0)

In [57]:
ColsToKeep.append('word_number')

on ne garde que les colonnes qui nous interessent

In [58]:
dfTemp = dfTemp.loc[:,ColsToKeep]

On groupe par order_id et on fait la somme des mots ainsi que la moyenne des scores

In [59]:
grouped = dfTemp.groupby(by= ['order_id'] ,dropna=False)
dfTemp = grouped.agg({'review_score':'mean','word_number':'sum'})
dfTemp = dfTemp.reset_index()
dfTemp.head()

,order_id,review_score,word_number
0,00010242fe8c5a6d1ba2dd792cb16214,5.0,6.0
1,00018f77f2f0320c557190d7a144bdd3,4.0,0.0
2,000229ec398224ef6ca0657da4fc703e,5.0,13.0
3,00024acbcdf0a6daa1e931b038114c75,4.0,0.0
4,00042b26cf59d7ce69dfabb4e55b4fd9,5.0,7.0


In [60]:
dfTemp.duplicated().value_counts()

False    98673
dtype: int64

Merge

In [61]:
df = pd.merge(df,dfTemp.loc[:,ColsToKeep], on='order_id',how='outer')
print("Shape for df is: {}".format(df.shape))
df.head()

Shape for df is: (103200, 13)


,customer_id,order_id,order_status,order_purchase_timestamp,order_delivered_customer_date,order_estimated_delivery_date,product_id,price,payment_value,customer_unique_id,product_category_name,review_score,word_number
0,9ef432eb6251297304e76186b10a928d,e481f51cbdc54678b7cc49136f2d6af7,delivered,2017-10-02 10:56:33,2017-10-10 21:25:13,2017-10-18 00:00:00,87285b34884572647811a353c7ac498a,29.99,38.71,7c396fd4830fd04220f754e42b4e5bff,utilidades_domesticas,4.0,32.0
1,b0830fb4747a6c6d20dea0b8c802d7ef,53cdb2fc8bc7dce0b6741e2150273451,delivered,2018-07-24 20:41:37,2018-08-07 15:27:45,2018-08-13 00:00:00,595fac2a385ac33a80bd5114aec74eb8,118.70,141.46,af07308b275d755c9edb36a90c618231,perfumaria,4.0,4.0
2,41ce2a54c0b03bf3443c3d931a367089,47770eb9100c2d0c44946d9cf07ec65d,delivered,2018-08-08 08:38:49,2018-08-17 18:06:29,2018-09-04 00:00:00,aa4383b373c6aca5d8797843e5594415,159.90,179.12,3a653a41f6f9fc3d2a113cf8398680e8,automotivo,5.0,0.0
3,f88197465ea7920adcdbec7375364d82,949d5b44dbf5de918fe9c16f97b45f8a,delivered,2017-11-18 19:28:06,2017-12-02 00:28:42,2017-12-15 00:00:00,d0b61bfb1de832b15ba9d266ca96e5b0,45.00,72.20,7c142cf63193a1473d2e66489a9ae977,pet_shop,5.0,20.0
4,8ab97904e6daea8866dbdbc4fb7aad2c,ad21c59c0840e6cb83a9ceb5573f8159,delivered,2018-02-13 21:18:39,2018-02-16 18:17:02,2018-02-26 00:00:00,65266b2da20d04dbe00c5c2d3bb7859e,19.90,28.62,72632f0f9dd73dfee390c9b22eb56dd6,papelaria,5.0,0.0


In [62]:
df.duplicated().value_counts()

False    103200
dtype: int64

Check for geolocation

In [63]:
isGeolocationIncluded = False
if isGeolocationIncluded == True:
    index = 7
    print("Shape for {} is {}".format(filesList[index].strip('olist'),dfList[index].shape))
    print("Cols are {}".format(dfList[index].columns))
    ColsToKeep = ['geolocation_zip_code_prefix']
    ColsToKeep.append('geolocation_lat')
    ColsToKeep.append('geolocation_lng')
    dfTemp = dfList[index].copy()
    display(dfTemp.head())

Merge

In [64]:
if isGeolocationIncluded == True:
    d = pd.merge(df, dfTemp.loc[:,ColsToKeep], how='left',left_on='customer_zip_code_prefix',
                 right_on='geolocation_zip_code_prefix')
    print("Shape for df is: {}".format(d.shape))
    display(d.head())

Check for Translation

In [65]:
index = 8
print("Shape for {} is {}".format(filesList[index].strip('olist'),dfList[index].shape))
print("Cols are {}".format(dfList[index].columns))
ColsToKeep = ['product_category_name']
ColsToKeep.append('product_category_name_english')
dfTemp = dfList[index].copy()
display(dfTemp.head())

Shape for product_category_name_translation.csv is (71, 2)
Cols are Index(['product_category_name', 'product_category_name_english'], dtype='object')


,product_category_name,product_category_name_english
0,beleza_saude,health_beauty
1,informatica_acessorios,computers_accessories
2,automotivo,auto
3,cama_mesa_banho,bed_bath_table
4,moveis_decoracao,furniture_decor


Merge

In [66]:
isTranslate = False
if isTranslate == True:
    df = pd.merge(df,dfTemp.loc[:,ColsToKeep], on='product_category_name',how='left')
    print("Shape for df is: {}".format(df.shape))
    df.head()

In [67]:
df.duplicated().value_counts()

False    103200
dtype: int64

### 2. Nettoyage du DataFrame

In [68]:
print(df.shape)
df.head()

(103200, 13)


,customer_id,order_id,order_status,order_purchase_timestamp,order_delivered_customer_date,order_estimated_delivery_date,product_id,price,payment_value,customer_unique_id,product_category_name,review_score,word_number
0,9ef432eb6251297304e76186b10a928d,e481f51cbdc54678b7cc49136f2d6af7,delivered,2017-10-02 10:56:33,2017-10-10 21:25:13,2017-10-18 00:00:00,87285b34884572647811a353c7ac498a,29.99,38.71,7c396fd4830fd04220f754e42b4e5bff,utilidades_domesticas,4.0,32.0
1,b0830fb4747a6c6d20dea0b8c802d7ef,53cdb2fc8bc7dce0b6741e2150273451,delivered,2018-07-24 20:41:37,2018-08-07 15:27:45,2018-08-13 00:00:00,595fac2a385ac33a80bd5114aec74eb8,118.70,141.46,af07308b275d755c9edb36a90c618231,perfumaria,4.0,4.0
2,41ce2a54c0b03bf3443c3d931a367089,47770eb9100c2d0c44946d9cf07ec65d,delivered,2018-08-08 08:38:49,2018-08-17 18:06:29,2018-09-04 00:00:00,aa4383b373c6aca5d8797843e5594415,159.90,179.12,3a653a41f6f9fc3d2a113cf8398680e8,automotivo,5.0,0.0
3,f88197465ea7920adcdbec7375364d82,949d5b44dbf5de918fe9c16f97b45f8a,delivered,2017-11-18 19:28:06,2017-12-02 00:28:42,2017-12-15 00:00:00,d0b61bfb1de832b15ba9d266ca96e5b0,45.00,72.20,7c142cf63193a1473d2e66489a9ae977,pet_shop,5.0,20.0
4,8ab97904e6daea8866dbdbc4fb7aad2c,ad21c59c0840e6cb83a9ceb5573f8159,delivered,2018-02-13 21:18:39,2018-02-16 18:17:02,2018-02-26 00:00:00,65266b2da20d04dbe00c5c2d3bb7859e,19.90,28.62,72632f0f9dd73dfee390c9b22eb56dd6,papelaria,5.0,0.0


In [69]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 103200 entries, 0 to 103199
Data columns (total 13 columns):
 #   Column                         Non-Null Count   Dtype  
---  ------                         --------------   -----  
 0   customer_id                    103200 non-null  object 
 1   order_id                       103200 non-null  object 
 2   order_status                   103200 non-null  object 
 3   order_purchase_timestamp       103200 non-null  object 
 4   order_delivered_customer_date  100195 non-null  object 
 5   order_estimated_delivery_date  103200 non-null  object 
 6   product_id                     102425 non-null  object 
 7   price                          102425 non-null  float64
 8   payment_value                  103199 non-null  float64
 9   customer_unique_id             103200 non-null  object 
 10  product_category_name          100965 non-null  object 
 11  review_score                   102384 non-null  float64
 12  word_number                   

Sauvegarde du dataframe avant nettoyage

In [70]:
dfClean = df.copy()

On supprime certaines colonnes

In [71]:
dfClean = dfClean.drop(['product_category_name','order_status'],axis=1)

In [72]:
dfClean.duplicated().sum()

0

somme des prix d'items pour une meme commande

In [73]:
# extract price to do sum by orders
pricedf = dfClean.loc[:,['order_id','price','payment_value'] ]
#a['order_id'].value_counts()
pricedf = pricedf.groupby(by=['order_id','payment_value'],as_index=False,dropna=False).sum()
#test = test.reset_index()
#test['order_id'].value_counts()
pricedf.rename(columns={'price':'somme_price'},inplace=True)
pricedf.head()

,order_id,payment_value,somme_price
0,00010242fe8c5a6d1ba2dd792cb16214,72.19,58.90
1,00018f77f2f0320c557190d7a144bdd3,259.83,239.90
2,000229ec398224ef6ca0657da4fc703e,216.87,199.00
3,00024acbcdf0a6daa1e931b038114c75,25.78,12.99
4,00042b26cf59d7ce69dfabb4e55b4fd9,218.04,199.90


selection des colonnes a garder

In [74]:
keep = getCompCols(dfClean,['price','product_id'])

In [75]:
dfClean = pd.merge(dfClean.loc[:,keep] ,pricedf.loc[:,['order_id','somme_price']], on='order_id',how='outer')
print("Shape for dfClean is: {}".format(dfClean.shape))
dfClean.head()

Shape for dfClean is: (103200, 10)


,customer_id,order_id,order_purchase_timestamp,order_delivered_customer_date,order_estimated_delivery_date,payment_value,customer_unique_id,review_score,word_number,somme_price
0,9ef432eb6251297304e76186b10a928d,e481f51cbdc54678b7cc49136f2d6af7,2017-10-02 10:56:33,2017-10-10 21:25:13,2017-10-18 00:00:00,38.71,7c396fd4830fd04220f754e42b4e5bff,4.0,32.0,29.99
1,b0830fb4747a6c6d20dea0b8c802d7ef,53cdb2fc8bc7dce0b6741e2150273451,2018-07-24 20:41:37,2018-08-07 15:27:45,2018-08-13 00:00:00,141.46,af07308b275d755c9edb36a90c618231,4.0,4.0,118.70
2,41ce2a54c0b03bf3443c3d931a367089,47770eb9100c2d0c44946d9cf07ec65d,2018-08-08 08:38:49,2018-08-17 18:06:29,2018-09-04 00:00:00,179.12,3a653a41f6f9fc3d2a113cf8398680e8,5.0,0.0,159.90
3,f88197465ea7920adcdbec7375364d82,949d5b44dbf5de918fe9c16f97b45f8a,2017-11-18 19:28:06,2017-12-02 00:28:42,2017-12-15 00:00:00,72.20,7c142cf63193a1473d2e66489a9ae977,5.0,20.0,45.00
4,8ab97904e6daea8866dbdbc4fb7aad2c,ad21c59c0840e6cb83a9ceb5573f8159,2018-02-13 21:18:39,2018-02-16 18:17:02,2018-02-26 00:00:00,28.62,72632f0f9dd73dfee390c9b22eb56dd6,5.0,0.0,19.90


In [76]:
dfClean.duplicated().sum()

3759

Comme on a oter les produits on a des doublons maintenant, donc on les enlèves

In [77]:
dfClean.drop_duplicates(inplace=True)
dfClean.shape

(99441, 10)

In [78]:
dfClean.reset_index(drop=True,inplace=True)

#### Valeurs manquantes

In [79]:
dfClean.isna().sum()

customer_id                         0
order_id                            0
order_purchase_timestamp            0
order_delivered_customer_date    2965
order_estimated_delivery_date       0
payment_value                       1
customer_unique_id                  0
review_score                      768
word_number                       768
somme_price                         0
dtype: int64

on remplace les dates de livraisons manquantes par celles de livraisons estimée

In [80]:
dfClean['order_delivered_customer_date'] = dfClean['order_delivered_customer_date'].fillna(dfClean['order_estimated_delivery_date'])

On attribut 0 mot si aucun commentaire n'à été laissé

In [81]:
dfClean['word_number'] = dfClean['word_number'].fillna(0)

On remplace les scores manquants par la moyenne des scores

In [82]:
dfClean['review_score'].mean(),dfClean['review_score'].median()

(4.0867934152875325, 5.0)

In [83]:
dfClean['review_score'] = dfClean['review_score'].fillna(dfClean['review_score'].mean())

Les paiements manquants sont remplacés par les couts de commandes

In [84]:
dfClean['payment_value'] = dfClean['payment_value'].fillna(dfClean['somme_price'])

In [85]:
dfClean.isna().sum()

customer_id                      0
order_id                         0
order_purchase_timestamp         0
order_delivered_customer_date    0
order_estimated_delivery_date    0
payment_value                    0
customer_unique_id               0
review_score                     0
word_number                      0
somme_price                      0
dtype: int64

### 3. Mise en forme des datas

In [86]:
dfClean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99441 entries, 0 to 99440
Data columns (total 10 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   customer_id                    99441 non-null  object 
 1   order_id                       99441 non-null  object 
 2   order_purchase_timestamp       99441 non-null  object 
 3   order_delivered_customer_date  99441 non-null  object 
 4   order_estimated_delivery_date  99441 non-null  object 
 5   payment_value                  99441 non-null  float64
 6   customer_unique_id             99441 non-null  object 
 7   review_score                   99441 non-null  float64
 8   word_number                    99441 non-null  float64
 9   somme_price                    99441 non-null  float64
dtypes: float64(4), object(6)
memory usage: 7.6+ MB


Supression du prix des articles maintenant

In [87]:
dfClean.drop('somme_price',axis=1,inplace=True)

Traitements des dates et conversions en jours

In [88]:
dfClean['order_delivered_customer_date'] = dfClean['order_delivered_customer_date'].apply(pd.to_datetime)
dfClean['order_purchase_timestamp'] = dfClean['order_purchase_timestamp'].apply(pd.to_datetime)

Introduction de la variable delai (difference entre date de livraison et date de la commande)

In [89]:
dfClean['delai'] = dfClean['order_delivered_customer_date'] - dfClean['order_purchase_timestamp']

In [90]:
dfClean['delai'] = dfClean['delai'].apply(lambda x: x.days)

Introduction de la variable retard (difference entre la variable de livraison et la date estimée). Le retard peut etre négatif si la livraison à lieu plus tot que prevu.

In [91]:
dfClean['retard'] = dfClean['order_delivered_customer_date'] - dfClean['order_estimated_delivery_date'].apply(pd.to_datetime)

In [92]:
dfClean['retard'] = dfClean['retard'].apply(lambda x: x.days)

Suppression des variables initiales

In [93]:
dfClean.drop(['order_delivered_customer_date','order_estimated_delivery_date'],axis=1,inplace=True)

In [94]:
dfClean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99441 entries, 0 to 99440
Data columns (total 9 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   customer_id               99441 non-null  object        
 1   order_id                  99441 non-null  object        
 2   order_purchase_timestamp  99441 non-null  datetime64[ns]
 3   payment_value             99441 non-null  float64       
 4   customer_unique_id        99441 non-null  object        
 5   review_score              99441 non-null  float64       
 6   word_number               99441 non-null  float64       
 7   delai                     99441 non-null  int64         
 8   retard                    99441 non-null  int64         
dtypes: datetime64[ns](1), float64(3), int64(2), object(3)
memory usage: 6.8+ MB


#### Sauvegarde brut du dataframe pour usage avec le notebook de simulation

In [95]:
path = "data/cleaned/"

dfClean.to_csv(path+outputRawfilename, index=False)

#### Sauvegarde du dataframe mis en forme pour usage avec le notebook d'essai

Mise en forme du dataframe total

In [96]:
maxDate = dfClean['order_purchase_timestamp'].max()
print("Most recent date is ",maxDate) 
dfReduct = extract(dfClean,maxDate)
dfReduct.head()

Most recent date is  2018-10-17 17:30:18
Calling extract with date :  2018-10-17 17:30:18
Number of observation selected 99441


,Frequency,Recency,retard,delai,Monetary,review_score,word_number
0,1,1818,-5.0,6.0,141.90,5.0,20.0
1,1,1821,-5.0,3.0,27.19,4.0,0.0
2,1,2244,-2.0,25.0,86.22,3.0,0.0
3,1,2028,-12.0,20.0,43.62,4.0,2.0
4,1,1995,-8.0,13.0,196.89,5.0,0.0


In [97]:
dfReduct.isna().sum()

Frequency       0
Recency         0
retard          0
delai           0
Monetary        0
review_score    0
word_number     0
dtype: int64

In [98]:
dfReduct.duplicated().sum()

53

In [99]:
dfReduct.nunique()

Frequency           9
Recency           631
retard            337
delai             253
Monetary        28876
review_score       45
word_number        88
dtype: int64

In [100]:
dfReduct.shape

(96096, 7)

#### Ecriture du dataframe final mis en forme

In [101]:
dfReduct.to_csv(path+outputFullfilename, index=False)